# Operators

> API details.

In [ ]:
#default_exp operators

In [ ]:
#export
from smpr3d.kernels import *
import torch as th
import numpy as np
import math as m
import numba.cuda as cuda
import torch.nn as nn
import cmath as cm
from numpy.fft import fftfreq 
th.autograd.set_detect_anomaly(True)

In [ ]:
#export 
def r2c(real_tensor):
    '''Convert from real to complex'''
    return real_tensor+0j

In [ ]:
#export 
def convolve_kernel(tensor_in, kernel, n_dim=1, flag_inplace=True):
    '''
    Compute convolution FFT(tensor_in) and kernel
    Required Args:
        tensor_in: variable 1 in real space
        kernel: variable 2 in reciprocal space

    Optional Args [default]
        n_dim: number of dimensions to compute convolution [1]
        flag_inplace: Whether or not compute convolution inplace, result saved in 'tensor_in' [True]
    '''
    dim = [-n_dim+i for i in range(n_dim)]
    if flag_inplace:
        tensor_in  = th.fft.fftn(tensor_in, dim=dim)
        tensor_in *= kernel
        tensor_in  = th.fft.ifftn(tensor_in, dim=dim)
        return tensor_in
    else:
        output  = th.fft.fftn(tensor_in, dim=dim)
        output *= kernel
        output  = th.fft.ifftn(output, dim=dim)
        return output
    
def generate_grid_1d(shape, pixel_size = 1, flag_fourier = False, dtype = th.float32, device = th.device('cuda')):
    """
    This function generates 1D Fourier grid, and is centered at the middle of the array
    Inputs:
        shape    - length of the array
        pixel_size      - pixel size
    Optional parameters:
        flag_fourier - flag indicating whether the final array is circularly shifted
                     should be false when computing real space coordinates
                     should be true when computing Fourier coordinates
    Outputs:
        x_lin       - 1D grid (real or fourier)

    """
    pixel_size = 1./pixel_size/shape if flag_fourier else pixel_size
    x_lin = (th.arange(shape, dtype=dtype, device=device) - shape//2) * pixel_size
    if flag_fourier:
        x_lin = th.roll(x_lin, -1 * int(shape)//2)
    return x_lin

def generate_grid_2d(shape, pixel_size = 1, flag_fourier = False, dtype = th.float32, device = th.device('cuda')):
    """
    This function generates 2D Fourier grid, and is centered at the middle of the array
    Inputs:
        shape              - shape of the grid (number of y pixels, number of x pixels)
        pixel_size         - pixel size
    Optional parameters:
        flag_fourier       - flag indicating whether the final array is circularly shifted
                             should be false when computing real space coordinates
                             should be true when computing Fourier coordinates
    Outputs:
        y_lin, x_lin       - 2D grid
    Usage:
        y_lin, x_lin = generate_grid_2d(...)

    """    
    assert len(shape) == 2, "shape should be two dimensional!"
    #recompute pixel size for fourier space sampling
    y_lin  = generate_grid_1d(shape[0], pixel_size, flag_fourier = flag_fourier, dtype=dtype, device=device)
    x_lin  = generate_grid_1d(shape[1], pixel_size, flag_fourier = flag_fourier, dtype=dtype, device=device)
    y_lin, x_lin = th.meshgrid(y_lin, x_lin)
    return y_lin, x_lin
def generate_hard_pupil(shape, pixel_size, numerical_aperture, wavelength, \
                   dtype=th.float32, device=th.device('cuda')):
    """
    This function generates pupil function(circular function) given shape, pixel_size, na, and wavelength
    """
    assert len(shape) == 2, "pupil should be two dimensional!"
    ky_lin, kx_lin = generate_grid_2d(shape, pixel_size, flag_fourier=True, dtype=dtype, device=device)

    pupil_radius = numerical_aperture/wavelength
    pupil        = (kx_lin**2 + ky_lin**2 <= pupil_radius**2).type(dtype)
    return op.r2c(pupil)

In [ ]:
#export 
def generate_angular_spectrum_kernel(shape, pixel_size, wavelength, \
                                     numerical_aperture=None,  flag_band_limited=True, \
                                     dtype=th.float32, device=th.device('cuda')):
    """
    Function that generates angular spectrum propagation kernel WITHOUT the distance
    The angular spectrum has the following form:
    p = exp(distance * kernel)
    kernel = 1j * 2 * pi * sqrt((ri/wavelength)**2-x**2-y**2)
    and this function generates the kernel only!
    """
    assert len(shape) == 2, "pupil should be two dimensional!"
    ky_lin, kx_lin = generate_grid_2d(shape, pixel_size, flag_fourier=True, dtype=dtype, device=device)
    if flag_band_limited:
        assert numerical_aperture is not None, "need to provide numerical aperture of the system!"
        pupil_crop    = r2c(generate_hard_pupil(shape, pixel_size, numerical_aperture, wavelength, dtype, device))
    else: 
        pupil_crop    = 1.0
    # prop_kernel = 2.0 * np.pi * pupil_crop * \
    #               ((1./wavelength)**2 - kx_lin**2 - ky_lin**2) ** 0.5
    prop_kernel = -1 * np.pi * wavelength * pupil_crop * (kx_lin**2 + ky_lin**2)
    return 1j *prop_kernel

In [ ]:
#export 
class Pupil(nn.Module):
    """
    Class for applying pupil in forward model computation
    """
    def __init__(self, shape, pixel_size, wavelength, \
                 numerical_aperture = 1.0, pupil = None, \
                 dtype=th.float32, device=th.device('cuda'), **kwargs):
        super(Pupil, self).__init__()
        if pupil is not None:
            self.pupil = pupil.type(dtype).to(device)
            if len(self.pupil.shape) == 2:
                self.pupil = r2c(self.pupil)
        else:
            self.pupil = generate_hard_pupil(shape, pixel_size, numerical_aperture, wavelength, dtype, device)
    def get_pupil(self):
        return self.pupil.cpu()
    def forward(self, field):
        field_out = convolve_kernel(field, self.pupil, 2, False)
        return field_out

MAX_DIM = 512*512*850
class ImageRotation:
    """
    A rotation class compute 3D rotation using FFT
    """
    def __init__(self, shape, axis = 0, pad = True, pad_value = 0, dtype = th.float32, device = th.device('cuda')):
        self.dim       = np.array(shape)
        self.axis      = axis
        self.pad_value = pad_value
        if pad:
            self.pad_size            = np.ceil(self.dim / 2.0).astype('int')
            self.pad_size[self.axis] = 0
            self.dim                += 2*self.pad_size
        else:
            self.pad_size  = np.asarray([0,0,0])
        
        self.dim          = [int(size) for size in self.dim]

        self.range_crop_y = slice(self.pad_size[0],self.pad_size[0] + shape[0])
        self.range_crop_x = slice(self.pad_size[1],self.pad_size[1] + shape[1])
        self.range_crop_z = slice(self.pad_size[2],self.pad_size[2] + shape[2])

        self.y            = generate_grid_1d(self.dim[0], dtype=dtype, device=device).unsqueeze(-1).unsqueeze(-1)
        self.x            = generate_grid_1d(self.dim[1], dtype=dtype, device=device).unsqueeze(0).unsqueeze(-1)
        self.z            = generate_grid_1d(self.dim[2], dtype=dtype, device=device).unsqueeze(0).unsqueeze(0)
        
        self.ky           = generate_grid_1d(self.dim[0], flag_fourier = True, dtype=dtype, device=device).unsqueeze(-1).unsqueeze(-1)
        self.kx           = generate_grid_1d(self.dim[1], flag_fourier = True, dtype=dtype, device=device).unsqueeze(0).unsqueeze(-1)
        self.kz           = generate_grid_1d(self.dim[2], flag_fourier = True, dtype=dtype, device=device).unsqueeze(0).unsqueeze(0)

        #Compute FFTs sequentially if object size is too large
        self.slice_per_tile = int(np.min([np.floor(MAX_DIM * self.dim[self.axis] / np.prod(self.dim)), self.dim[self.axis]]))            
        self.dtype          = dtype
        self.device         = device

        if self.axis == 0:
            self.coord_phase_1 = -2.0 * np.pi * self.kz * self.x
            self.coord_phase_2 = -2.0 * np.pi * self.kx * self.z
        elif self.axis == 1:
            self.coord_phase_1 = -2.0 * np.pi * self.kz * self.y
            self.coord_phase_2 = -2.0 * np.pi * self.ky * self.z
        elif self.axis == 2:
            self.coord_phase_1 = -2.0 * np.pi * self.kx * self.y
            self.coord_phase_2 = -2.0 * np.pi * self.ky * self.x

    def _rotate_3d(self, obj, shear_phase_1, shear_phase_2):
        """
        This function rotates a 3D image by shearing, (applied in Fourier space)
        ** Note: the rotation is performed along the z axis

        [ cos(theta)  -sin(theta) ] = [ 1  alpha ] * [ 1     0  ] * [ 1  alpha ]
        [ sin(theta)  cos(theta)  ]   [ 0    1   ]   [ beta  1  ]   [ 0    1   ]
        alpha = tan(theta/2)
        beta = -sin(theta)

        Shearing in one shapeension is applying phase shift in 1D fourier transform
        Input:
          obj: 3D array (supposed to be an image), the axes are [z,y,x]
          theta: desired angle of rotation in *degrees*
        Output:
          obj_rotate: rotate 3D array
        """
        flag_complex = obj.is_complex()
        self.obj_rotate[self.range_crop_y, self.range_crop_x, self.range_crop_z] = r2c(obj)
        if self.axis == 0:
            self.obj_rotate = convolve_kernel(self.obj_rotate, shear_phase_1) #y,x,z
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_2.permute([0,2,1])) #y,z,x
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_1) #y,x,z

        elif self.axis == 1:
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([1,0,2]), shear_phase_1.permute([1,0,2])) #x,y,z
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_2.permute([1,2,0])) #x,z,y
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_1.permute([1,0,2])) #x,y,z
            self.obj_rotate = self.obj_rotate.permute([1,0,2])

        elif self.axis == 2:
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([2,0,1]), shear_phase_1.permute([2,0,1])) #z,y,x
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_2.permute([2,1,0])) #z,x,y
            self.obj_rotate = convolve_kernel(self.obj_rotate.permute([0,2,1]), shear_phase_1.permute([2,0,1])) #z,y,x
            self.obj_rotate = self.obj_rotate.permute([1,2,0])
        if flag_complex:
            obj[:] = self.obj_rotate[self.range_crop_y, self.range_crop_x, self.range_crop_z]
        else:
            obj[:] = self.obj_rotate[self.range_crop_y, self.range_crop_x, self.range_crop_z].real
        return obj

    def forward(self, obj, theta):
        self.theta = theta
        if theta == 0:
            return obj
        else:
            flag_cpu = False
            if self.device == th.device('cuda'):
                if not obj.is_cuda:
                    flag_cpu = True
            #         obj = obj.to(self.device)
            theta      *= np.pi / 180.0
            alpha       = 1.0 * np.tan(theta / 2.0)
            beta        = np.sin(-1.0 * theta)

            shear_phase_1 = th.exp(1j * self.coord_phase_1 * alpha)
            shear_phase_2 = th.exp(1j * self.coord_phase_2 * beta)

            self.dim[self.axis] = self.slice_per_tile
            self.obj_rotate = r2c(th.ones([self.dim[0], self.dim[1], self.dim[2]], dtype=self.dtype, device=self.device) * self.pad_value)

            for idx_start in range(0, obj.shape[self.axis], self.slice_per_tile):
                idx_end = np.min([obj.shape[self.axis], idx_start+self.slice_per_tile])
                idx_slice = slice(idx_start, idx_end)
                self.dim[self.axis] = int(idx_end - idx_start)
                if self.axis == 0:
                    self.range_crop_y = slice(0, self.dim[self.axis])
                    obj[idx_slice,:,:] = self._rotate_3d(obj[idx_slice,:,:].cuda(), shear_phase_1, shear_phase_2).cpu()
                elif self.axis == 1:
                    self.range_crop_x = slice(0, self.dim[self.axis])
                    obj[:,idx_slice,:] = self._rotate_3d(obj[:,idx_slice,:].cuda(), shear_phase_1, shear_phase_2).cpu()
                elif self.axis == 2:
                    self.range_crop_z = slice(0, self.dim[self.axis])
                    obj[:,:,idx_slice] = self._rotate_3d(obj[:,:,idx_slice].cuda(), shear_phase_1, shear_phase_2).cpu()
                self.obj_rotate[:] = self.pad_value + 0.j
            self.dim[self.axis] = obj.shape[self.axis]
            self.obj_rotate = None
            if self.device == th.device('cuda'):
                th.cuda.empty_cache()
            if flag_cpu:
                obj = obj.cpu()
            return obj

    def backward(self, obj):
        theta = -1 * self.theta
        if theta == 0:
            return obj
        else:
            if self.device == th.device("cuda"):
                if not obj.is_cuda:
                    obj = obj.to(self.device)
            theta      *= np.pi / 180.0
            alpha       = 1.0 * np.tan(theta / 2.0)
            beta        = np.sin(-1.0 * theta)
            
            shear_phase_1 = th.exp(1j * self.coord_phase_1 * alpha)
            shear_phase_2 = th.exp(1j * self.coord_phase_2 * beta)

            self.dim[self.axis] = self.slice_per_tile
            self.obj_rotate = r2c(th.zeros([self.dim[0], self.dim[1], self.dim[2]], dtype=self.dtype, device=self.device))

            for idx_start in range(0, obj.shape[self.axis], self.slice_per_tile):
                idx_end = np.min([obj.shape[self.axis], idx_start+self.slice_per_tile])
                idx_slice = slice(idx_start, idx_end)
                self.dim[self.axis] = int(idx_end - idx_start)
                if self.axis == 0:
                    self.range_crop_y = slice(0, self.dim[self.axis])
                    obj[idx_slice,:,:] = self._rotate_3d(obj[idx_slice,:,:], alpha, beta, shear_phase_1, shear_phase_2)
                elif self.axis == 1:
                    self.range_crop_x = slice(0, self.dim[self.axis])
                    obj[:,idx_slice,:] = self._rotate_3d(obj[:,idx_slice,:], alpha, beta, shear_phase_1, shear_phase_2)
                elif self.axis == 2:
                    self.range_crop_z = slice(0, self.dim[self.axis])
                    obj[:,:,idx_slice] = self._rotate_3d(obj[:,:,idx_slice], alpha, beta, shear_phase_1, shear_phase_2)
                self.obj_rotate[:] = 0.0
            self.dim[self.axis] = obj.shape[self.axis]
            self.obj_rotate = None         
            if not obj.is_cuda:
                obj = obj.cpu()
            return obj

In [ ]:
#export 
def calc_psi(r, t, z, out):
    out[:] = 0
    K = r.shape[0]
    MY, MX = out.shape
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(K * MY * MX / threadsperblock)
    psi_kernel[blockspergrid, threadsperblock](r, t, z, out)
    return out



In [ ]:
#export 
class ComplexAbs(th.autograd.Function):
    '''Absolute value class for autograd'''
    @staticmethod
    def forward(ctx, tensor_in):
        output = th.abs(tensor_in)
        ctx.save_for_backward(th.angle(tensor_in))
        return output

    @staticmethod
    def backward(ctx, grad_output):
        tensor_in_angle,     = ctx.saved_tensors
        return 0.5*th.exp(1j * tensor_in_angle) * grad_output


In [ ]:
#export 
class BinObject(th.autograd.Function):
    '''
    Class that bins the object along the direction of beam propagation (z)
    inputs:
    obj_in: input object 
    factor: factor at which the object will be binned
    '''
    @staticmethod
    def forward(ctx, obj_in, factor):
        assert (obj_in.shape[2] % factor) == 0
        assert len(obj_in.shape) == 3
        ctx.factor = factor
        if factor == 1:
            return obj_in
        n_y, n_x, n_z = obj_in.shape
        obj_out = obj_in.reshape(n_y, n_x, n_z//factor, factor).sum(3)
        return obj_out

    @staticmethod
    def backward(ctx, grad_output):
        factor = ctx.factor
        if factor == 1:
            return grad_output, None

        return grad_output.repeat_interleave(factor, dim=-1), None

In [ ]:
#export 
def AtF2(z, psi, r, out):
    """

    :param z:   K x MY x MX
    :param psi: B x K x MY x MX
    :param r:   K x 2
    :param out: B x NY x NX
    :return:
    """
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(z.shape) / threadsperblock)
    AtF2_kernel[blockspergrid, threadsperblock](z, psi, r, out)
    return out

In [ ]:
#export 
def dS(z, z_old, psi, psi_int, psi_int_max, r, out, alpha=0.1):
    """
    :param z:           K x MY x MX
    :param z_old:       K x MY x MX
    :param psi:         B x K x MY x MX
    :param psi_int:     B x K x MY x MX
    :param psi_int_max: B x K
    :param r:           K x 2
    :param out:         B x NY x NX
    :param alpha:       float
    :return: 
    """

    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(z.shape) / threadsperblock)
    dS_kernel[blockspergrid, threadsperblock](z, z_old, psi, psi_int, psi_int_max, alpha, r, th.view_as_real(out))
    return out

In [ ]:
#export 
def A_realspace(r, t, psi, out):
    """

    :param r:   K x 2
    :param t:   BB x NY x NX
    :param psi: B x K x MY x MX
    :param out: K x MY x MX
    :return:
    """
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(out.shape) / threadsperblock)
    # print(r.shape,t.shape,psi.shape,out.shape)
    A_realspace_kernel[blockspergrid, threadsperblock](r, th.view_as_real(t), th.view_as_real(psi), th.view_as_real(out))
    return out

In [ ]:
#export 

def calc_psi_denom(r, t, out):
    """

    :param r: K x 2
    :param t: BB x NY x NX
    :param out: BB x MY x MX
    :return:
    """

    out[:] = 0
    K = r.shape[0]
    BB, MY, MX = out.shape
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(BB * K * MY * MX / threadsperblock)
    psi_denom_kernel[blockspergrid, threadsperblock](r, t, out)
    return out

In [ ]:
#export 
def Qoverlap_real2(r, z, out):
    """

    :param r: K x 2
    :param z: BB x K x MY x MX
    :param out: BB x NY x NX
    :return: out
    """
    BB = out.shape[0]
    K = r.shape[0]
    out[:] = 1
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(BB * K * np.prod(z.shape) / threadsperblock)
    overlap_kernel_real2[blockspergrid, threadsperblock](r, z, out)
    return out

In [ ]:
#export 
def split(S, r, MY, MX):
    """
    
    :param S:  B x NY x NX
    :param r:  K x 2
    :param MY: int
    :param MX: int
    :return: 
    """
    B = S.shape[0]
    K, _ = r.shape
    out = th.zeros((B, K, MY, MX), dtype=th.complex64, device=S.device)
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(out.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    split_kernel4[blockspergrid, threadsperblock, stream](S, r, out)
    return out
    

In [ ]:
#export 
def A(S, Psi, r, r_min, out=None, Mx=0, My=0):
    """ Fastest version, takes precomputed phase factors, assumes S-matrix with beam tilt included

    :param S:               B x NY x NX
    :param phase_factors:   B x D x K x 2
    :param r:               D x K x 2
    :param out:             D x K x MY x MX
    :return: exit waves in out
    """
    B = S.shape[0]
    if out is None and My > 0 and Mx > 0:
        D, K, _ = r.shape
        out = th.zeros((D, K, My, Mx, 2), dtype=th.float32, device=S.device)
    else:
        out[:] = 0
    D, K, MY, MX, _ = out.shape
    gpu = cuda.get_current_device()
    threadsperblock = 128#gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array((B, MX, MY))) / threadsperblock)
    smatrix_forward_kernel[blockspergrid, threadsperblock, th.cuda.current_stream().cuda_stream]\
        (th.view_as_real(S), Psi.phase_factors, r, r_min, out)
    return th.view_as_complex(out)

@th.jit.script
def complex_matmul(a, b):
    """
    Complex matrix multiplication of tensors a and b.

    Pass conjugate = True to conjugate tensor b in the multiplication.
    """
    are, aim = th.unbind(a, -1)
    bre, bim = th.unbind(b, -1)
    real = are @ bre - aim @ bim
    imag = are @ bim + aim @ bre
    return th.stack([real, imag], -1)

def A_fast_full2(S, phase_factors, r, r_min, MY, MX):
    """ Fastest version, takes precomputed phase factors, assumes S-matrix with beam tilt included

    :param S:               B x NY x NX
    :param phase_factors:   K x B x 2
    :param r:               K x 2
    :param out:             K x MY x MX x 2
    :return: exit waves in out
    """
    B = S.shape[0]
    K, _ = r.shape
    out = th.zeros((K, MY, MX, B, 2), dtype=th.float32, device=S.device)
    K, MYMX, _, _, _ = out.shape
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = 256  # gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array((K, MX, MY, B))) / threadsperblock)

    phase_factors2 = phase_factors.unsqueeze(2)
    # 1 - get crops from S-matrix
    split_kernel[blockspergrid, threadsperblock, stream](th.view_as_real(S), r, out)
    out = out.view((K, MY * MX, B, 2))
    # 2 - complex batched matmul: K x MY*MX x B x 2 @ K x B x 1 x 2
    # print(out.shape)
    # print(phase_factors2.shape)
    exitwaves = complex_matmul(out, phase_factors2)
    # 3 - reshape
    exitwaves = exitwaves.view((K, MY, MX, 2))
    return exitwaves

def A_fast_full3(S, phase_factors, r, r_min, MY, MX):
    """ Fastest version, takes precomputed phase factors, assumes S-matrix with beam tilt included

    :param S:               B x NY x NX
    :param phase_factors:   K x B 
    :param r:               K x 2
    :param out:             K x MY x MX  
    :return: exit waves in out
    """
    B = S.shape[0]
    K, _ = r.shape
    out = th.zeros((K, MY, MX, B, 2), dtype=th.float32, device=S.device)
    K, MYMX, _, _, _ = out.shape
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    
    threadsperblock = 128  # gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array((K, MX, MY, B))) / threadsperblock)
    # 1 - get crops from S-matrix
    split_kernel[blockspergrid, threadsperblock, stream](th.view_as_real(S), r, out)
    
    # threadsperblock = 128  # gpu.MAX_THREADS_PER_BLOCK
    # blockspergrid = m.ceil(np.prod(np.array((K, B))) / threadsperblock)
    # # 1 - get crops from S-matrix
    # split_kernel2[blockspergrid, threadsperblock, stream](th.view_as_real(S), r, out)
    
    out = out.view((K, MY * MX, B, 2))
    out = th.view_as_complex(out)
    # 1.5 - convert to cupy
    # 2 - complex batched matmul: K x MY*MX x B @ K x B x 1
    # print(out.shape)
    # print(phase_factors2.shape)
    # print(out.dtype)
    # print(phase_factors2.dtype)
    phase_factors2 = phase_factors.unsqueeze(2)
    exitwaves = out @ phase_factors2
    # 3 - reshape
    exitwaves = exitwaves.view((K, MY, MX))
    #4 convert to pytorch
    return exitwaves

def A_fast_full5(S, phase_factors, r, r_min, MY, MX):
    """ Fastest version, takes precomputed phase factors, assumes S-matrix with beam tilt included

    :param S:               B x NY x NX
    :param phase_factors:   K x B 
    :param r:               K x 2
    :param out:             K x MY x MX  
    :return: exit waves in out
    """
    B = S.shape[0]
    K, _ = r.shape
    out = th.zeros((K, B, MY, MX), dtype=th.complex64, device=S.device)
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(np.array((K, B, MY, MX))) / threadsperblock)
    # 1 - get crops from S-matrix
    split_kernel4[blockspergrid, threadsperblock, stream](S, r, out)
    # 2 - complex batched matmul: K x 1 x B @ K x B x MY*MX --> K x 1 x MY * MX
    exitwaves = phase_factors.unsqueeze(1) @ out.view((K, B, MY * MX))
    # 3 - reshape
    return exitwaves.view((K, MY, MX))

def A_fast_full4(S, phase_factors, r, r_min, out=None, Mx=0, My=0):
    """ Fastest version, takes precomputed phase factors, assumes S-matrix with beam tilt included

    :param S:               B x NY x NX
    :param phase_factors:   B x D x K x 2
    :param r:               D x K x 2
    :param out:             D x K x MY x MX
    :return: exit waves in out
    """
    B = S.shape[0]
    if out is None and My > 0 and Mx > 0:
        D, K, _ = r.shape
        out = th.zeros((D, K, My, Mx, 2), dtype=th.float32, device=S.device)
    D, K, MY, MX, _ = out.shape
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    tbp = int(gpu.MAX_THREADS_PER_BLOCK**(1/3))

    # max dim of thread block (1024,1024,64), with a total of 1024 max
    # max dim of grid (2^32-1 , 2^16-1, 2^16-1)
    #                 (2^32-1 , 65535, 65535)

    threadsperblock = (tbp, tbp, tbp)
    blockspergrid = tuple(np.ceil((K/tbp, MY/tbp, MX/tbp)).astype(np.int32))# m.ceil(np.prod(np.array((B, MX, MY))) / threadsperblock)
    smatrix_forward_kernel_fast_full4[blockspergrid, threadsperblock, stream](th.view_as_real(S), phase_factors, r, r_min, out)
    return th.view_as_complex(out)

In [ ]:
#export 
from smpr3d.kernels import smatrix_backward_kernel_S
def AH_S(z, Psi, r, r_min, out=None, tau = th.tensor([1.0]), Ny=-1, Nx=-1):
    """
    Adjoint S-matrix operator for the full S-matrix. Expects pre-computed phase-factors as inputs.

    :param z:               D x K x My x Mx
    :param Psi:             D x My x Mx
    :param r:               D x K x 2
    :param r_min:           2
    :param out:             B x NY x NX x 2
    :param Ny:              optional, int
    :param Nx:              optional, int
    :return: result of adjoint S-matrix operator, shape (B x NY x NX x 2)
    """
    D, K, MY, MX = z.shape
    B = Psi.phase_factors.shape[0]

    if out is None and Ny > 0 and Nx > 0:
        out_is_gradient = True
        out = th.zeros((B, Ny, Nx, 2), dtype=th.float32, device=z.device)
    else:
        out_is_gradient = False
        tau /= (K * D)

    # shape D
    mean_probe_intensities = th.norm(Psi, p=2, dim=(1, 2))
    mean_probe_intensities /= MX * MY

    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = 128#gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array((B, MY, MX))) / threadsperblock)
    smatrix_backward_kernel_S[blockspergrid, threadsperblock, stream] \
        (th.view_as_real(z), Psi.phase_factors, mean_probe_intensities, r, r_min, out, tau)

    if out_is_gradient:
        out /= (K * D)

    return th.view_as_complex(out)

In [ ]:
#export 
from smpr3d.kernels import phase_factor_kernelDBK
def smatrix_phase_factorsBDK(Psi, r, take_beams, q, B, out=None):
    """
    Abbreviations:
    B: number of (input) beams in S-matrix
    D: number of scans/ aperture functions
    K: number of scan positions
    MY/MX: detector shape
    NY/NX: S-matrix shape

    :param Psi: q           D x B
    :param r:               D x K x 2
    :param take_beams:      MY x MX
    :param q:               2 x MY x MX
    :param out:             B x D x K x 2
    :return:
    """
    if out is None:
        D, K, _ = r.shape
        out = th.zeros((B, D, K, 2), dtype=th.float32, device=Psi.device)
    else:
        out[:] = 0
        _, D, K, c = out.shape

    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = 128#gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array(out.shape[:-1])) / threadsperblock)
    tb = take_beams[None, ...].expand(*Psi.shape)
    Psi_DB = Psi[tb].reshape(D, B)
    tb = take_beams[None, ...].expand(*q.shape)
    qB = q[tb].reshape(2, B)
    phase_factor_kernelDBK[blockspergrid, threadsperblock, stream](th.view_as_real(Psi_DB), r, qB, out)
    return out

In [ ]:
#export 
from smpr3d.kernels import phase_factor_kernelKB
def smatrix_phase_factorsKB(Psi, r, take_beams, q, q_indices, B, out=None):
    """
    Abbreviations:
    B: number of (input) beams in S-matrix
    K: number of scan positions
    MY/MX: detector shape
    NY/NX: S-matrix shape

    :param Psi: q           MY x MX
    :param r:               K x 2
    :param take_beams:      MY x MX
    :param q:               2 x MY x MX
    :param q_indices:       2 x MY x MX
    :param out:             K x B x 2
    :return:
    """
    if out is None:
        K, _ = r.shape
        out = th.zeros((K, B, 2), dtype=th.float32, device=Psi.device)
    else:
        out[:] = 0
        K, B, c = out.shape

    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = 128#gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(np.array(out.shape[:-1])) / threadsperblock)
    tb = take_beams.expand(*Psi.shape)
    Psi_B = Psi[tb].reshape(B)
    tb = take_beams[None, ...].expand(*q_indices.shape)
    qB = q[tb].reshape(2, B)
    phase_factor_kernelKB[blockspergrid, threadsperblock, stream](th.view_as_real(Psi_B), r, qB, out)
    return out

In [ ]:
#export

def gradient_fourier_2d(x):
    """

    :param x: B x K x MY x MX
    :return: 2 x B x K x MY x MX
    """
    qy, qx = th.meshgrid([th.fft.fftfreq(x.shape[-2]), th.fft.fftfreq(x.shape[-1])])
    # 2 x MY x MX
    q = th.stack([qy, qx]).to(x.device)
    for i in range(len(x.shape) - 2):
        q = q.unsqueeze(1)
    # print('x.shape, q.shape', x.shape, q.shape)
    dx_dr = th.fft.ifft2(th.fft.fft2(x, norm='ortho')[None, ...] * 2j * np.pi * q, norm='ortho')
    return dx_dr

class SubpixShift():
    def __init__(self, MY, MX, device):
        qy, qx = np.meshgrid(fftfreq(MY), fftfreq(MX), indexing='ij')
        q = th.stack([th.as_tensor(qy, dtype=th.float32, device=device),
                      th.as_tensor(qx, dtype=th.float32, device=device)])
        self.qqy = q[0][None, None, ...]
        self.qqx = q[1][None, None, ...]

    def __call__(self, w, rs):
        ramp = th.exp(-2j * np.pi * (self.qqy * rs[:, 0][:, None, None] + self.qqx * rs[:, 1][:, None, None]))
        Psi = th.fft.fft2(w, norm='ortho')
        # B x K x MY x MX
        # print(ramp.shape,Psi.shape)
        if len(Psi.shape) < 4:
            Psi = Psi.unsqueeze(1)
            Psi = Psi.repeat(1, ramp.shape[1], 1, 1)
        # B x K x MY x MX
        w = th.fft.ifft2(Psi * ramp, norm='ortho')
        return w

from typing import Union
from smpr3d.data import Sparse3DData
def prox_D_gaussian(
        z : th.tensor, 
        z_hat : th.tensor, 
        a : Union[th.tensor, Sparse3DData], 
        beta : float):
    """
    Proximal operator of the Gaussian log-likelihood.

    :param z:           K x My x Mx x 2, updated exit waves
    :param z_hat:       K x My x Mx x 2, current model exit waves
    :param a:           K x My x Mx,     measured amplitudes
    :param beta:        float                hyperparameter

    :return: z
    """
    # print(type(a))
    gpu = cuda.get_current_device()
    if isinstance(a, th.Tensor):
        threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
        blockspergrid = m.ceil(np.prod(np.array(a.shape)) / threadsperblock)
        stream = th.cuda.current_stream().cuda_stream
        loss = th.zeros((z.shape[0],), device=z.device, dtype=th.float32)
        prox_D_gaussian_kernel[blockspergrid, threadsperblock, stream](z, z_hat, a, loss, beta)
        loss = loss.cpu().numpy()
    elif isinstance(a, Sparse3DData):
        # print(a.indices.device,a.indices.dtype)
        # print(a.counts.device,a.counts.dtype)
        z, loss = sparse_amplitude_prox(z, z_hat, a.indices, a.counts, beta)
    return z, loss

def sparse_amplitude_prox(z, z_hat, indices_target, counts_target, beta):
    """
    Amplitude loss proximal operator

    :param z:                   K x M1 x M2
    :param z_hat:               K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :return: z (K x M1 x M2), loss (K,),
    """

    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(np.prod(z.shape)) / threadsperblock).astype(np.int32))

    loss = th.zeros((z.shape[0],), device=z.device, dtype=th.float32)
    no_count_indicator = th.iinfo(indices_target.dtype).max

    sparse_amplitude_prox_kernel[blockspergrid, threadsperblock](z, z_hat, indices_target, counts_target, loss,
                                                                 no_count_indicator, beta)
    return z, loss.cpu().numpy()

def Qoverlap(r, z, out):
    """
    :param r:       K x 2
    :param z:       B x K x MY x MX
    :param out:     B x NY x NX
    :return:
    """

    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(z.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    # overlap_kernel2[blockspergrid, threadsperblock, stream](r, th.view_as_real(z), th.view_as_real(out))
    overlap_kernel2[blockspergrid, threadsperblock, stream](r, z, out)
    return out

def sparse_smooth_truncated_amplitude_prox(z_model, z_hat, indices_target, counts_target, frame_dimensions, eps=0.5,
                                           lam=6e-1):
    """
    Smooth truncated amplitude loss from Chang et al., Overlapping Domain Decomposition Methods for Ptychographic Imaging, (2020)

    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (K,), grad (K x M1 x M2)
    """

    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(np.prod(z_model.shape)) / threadsperblock).astype(np.int32))

    loss = th.zeros((z_model.shape[0],), device=z_model.device, dtype=th.float32)
    grad = th.ones_like(z_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max

    sparse_smooth_truncated_amplitude_prox_kernel[blockspergrid, threadsperblock](z_model, indices_target,
                                                                                  no_count_indicator, eps, lam)
    return loss, grad


def init_z(I_cts, I_inds, Psi_init, out):
    """

    :param I_cts:    K x cts       , integer
    :param I_inds:   K x cts       , long
    :param Psi_init: MY x MX   , complex64
    :param out:      K x MY x MX   , complex64
    :return:
    """

    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(np.prod(out.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    no_count_indicator = th.iinfo(I_inds.dtype).max
    init_z_kernel[blockspergrid, threadsperblock, stream](I_cts, I_inds, Psi_init, no_count_indicator, out)
    return out

def Qoverlap_real(r, z, out):
    """

    :param r:   K x 2
    :param z:   B x K x MY x MX
    :param out: B x NY x NX
    :return:
    """
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(z.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    overlap_kernel_real[blockspergrid, threadsperblock, stream](r, z, out)
    return out

def Qoverlap_real2(r, z, out):
    K = r.shape[0]
    out[:] = 1
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(K * np.prod(z.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    overlap_kernel_real2[blockspergrid, threadsperblock, stream](r, z, out)
    return out

def Qsplit(r, t, out):
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(out.shape) / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    split_kernel[blockspergrid, threadsperblock, stream](r, t, out)
    return out

def calc_psi_denom(r, t, out):
    """

    :param r:   K x 2
    :param t:   B x NY x NX
    :param out: B x MY x MX
    :return:
    """
    out[:] = 0
    K = r.shape[0]
    B, MY, MX = out.shape
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK
    blockspergrid = m.ceil(B * K * MY * MX / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    psi_denom_kernel[blockspergrid, threadsperblock, stream](r, t, out)
    return out

def calc_psi(r, t, z, out):
    """

    :param r:   K x 2
    :param t:   B x NY x NX
    :param z:   K x MY x MX
    :param out: B x MY x MX
    :return:
    """
    out[:] = 0
    K = r.shape[0]
    B, MY, MX = out.shape
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(B * K * MY * MX / threadsperblock)
    stream = th.cuda.current_stream().cuda_stream
    psi_kernel[blockspergrid, threadsperblock, stream](r, t, z, th.view_as_real(out))
    return out

def A_realspace(r, t, psi, out):
    """

    :param r:   K x 2
    :param t:   BB x NY x NX
    :param psi: B x K x MY x MX
    :param out: K x MY x MX
    :return:
    """
    gpu = cuda.get_current_device()
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(out.shape) / threadsperblock)
    # print(r.shape,t.shape,psi.shape,out.shape)
    A_realspace_kernel[blockspergrid, threadsperblock](r, th.view_as_real(t), th.view_as_real(psi),
                                                       th.view_as_real(out))
    return out

def A(t, psi, r, out):
    """

    :param t:   B x NY x NX
    :param psi: B x K x MY x MX
    :param r:   K x 2
    :param out: K x MY x MX
    :return:
    """

    Qsplit(r, t, out)
    out *= psi
    out = th.fft.fft2(out, norm='ortho')
    return out

def At(z, psi, r, out):
    """

    :param z:    K x MY x MX
    :param psi:  B x K x MY x MX
    :param r:    K x 2
    :param out:  B x NY x NX
    :return:
    """
    out[:] = 0
    return Qoverlap(r, th.conj(psi) * th.fft.ifft2(z, norm='ortho'), out)

def AtF(z, psi, r, out):
    """
    AtF(z_hat, psi, r, S_model_tmp)
    :param z:       K x MY x MX
    :param psi:     B x K x MY x MX
    :param r:       K x 2
    :param out:     B x NY x NX
    :return:
    """
    return Qoverlap(r, th.conj(psi) * z, out)